In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
%matplotlib inline

## Split original videos into individual frames

In [ ]:
original_video_dir = "../original_videos"
original_video_names = [
    "original_video_1.mp4",
    "original_video_2.mp4",
    "original_video_3.mp4",
    "original_video_4.mp4",
    "original_video_5.mp4",
    "original_video_6.mp4",
]

In [ ]:
def split_video(read_dir, file_name, write_dir):
    command = "ffmpeg -i " + os.path.join(read_dir, file_name) + " -r 30 " + \
               os.path.join(write_dir, "image_%03d.png")
    print(command)
    os.system(command)

In [ ]:
for original_video in original_video_names:
    write_dir = os.path.join(original_video_dir, original_video.split(".")[0])
    if not os.path.exists(write_dir):
        os.makedirs(write_dir)
    split_video(original_video_dir, original_video, write_dir)

## Keep selection of frames for evaluation and rename

In [ ]:
frame_ranges = {
    "original_video_1.mp4": (500, 899),
    "original_video_2.mp4": (100, 299),
    "original_video_3.mp4": (580, 879),
    "original_video_4.mp4": (420, 519),
    "original_video_5.mp4": (230, 329),
    "original_video_6.mp4": (160, 359),
}

total_num_frames = 0
for k in frame_ranges:
    tup = frame_ranges[k]
    total_num_frames += tup[1] - tup[0] + 1
    
crop_directions = {
    "original_video_1.mp4": "left",
    "original_video_2.mp4": "left",
    "original_video_3.mp4": "right",
    "original_video_4.mp4": "middle",
    "original_video_5.mp4": "left",
    "original_video_6.mp4": "right",
}

In [ ]:
def crop_frames_and_save(read_dir, write_dir, frame_range, crop_dir):
    if not os.path.exists(write_dir):
        os.makedirs(write_dir)
    
    start_index = frame_range[0]
    end_index = frame_range[1] + 1
    curr_index = 1
    
    for i in range(start_index, end_index):
        image = Image.open(os.path.join(read_dir, "image_" + format(i, "03d") + ".png"))
        image = image.convert("RGB")
        image = np.array(image)
        if crop_dir == "left":
            cropped = image[300:650, :780, :]
        elif crop_dir == "right":
            cropped = image[300:650, 500:, :]
        else:
            cropped = image[300:650, 250:1030, :]
        plt.imsave(os.path.join(write_dir, "image_" + format(curr_index, "03d") + ".png"), cropped)
        curr_index += 1

In [ ]:
processed_dirs = []
for k in frame_ranges:
    frame_range = frame_ranges[k]
    crop_dir = crop_directions[k]
    
    read_dir = os.path.join("../original_videos", k.split(".")[0])
    write_dir = os.path.join("../dataset", k.split(".")[0].split("_")[-1])
    processed_dirs.append(write_dir)
    
    crop_frames_and_save(read_dir, write_dir, frame_range, crop_dir)

In [ ]:
k = "original_video_3.mp4"
frame_range = frame_ranges[k]
crop_dir = crop_directions[k]

read_dir = os.path.join("../original_videos", k.split(".")[0])
write_dir = os.path.join("../dataset_2", k.split(".")[0].split("_")[-1])

crop_frames_and_save(read_dir, write_dir, frame_range, crop_dir)

## Recombine processed frames into video

In [ ]:
def combine_video_libx264(read_dir, write_dir, write_file_name):
    command = "ffmpeg -r 30 -i " + os.path.join(read_dir, "image_%03d.png") + " -vcodec libx264 " + \
               os.path.join(write_dir, write_file_name)
    print(command)
    os.system(command)

In [ ]:
write_dir = "../dataset"
for read_dir in processed_dirs:
    combine_video_libx264(read_dir, write_dir, read_dir.split("/")[-1] + ".mp4")